In [7]:
include("src/Models.jl")
include("src/Params.jl")
include("src/Utils.jl")
using DifferentialEquations
using Plots; gr()
using Optimization
using OptimizationOptimJL
using Zygote

In [86]:
# params

A0 = 1.0
fATP = 1.0
p_rest = [kUTA, kTU, kTUA, kTDA, kDT, kDTA, kDS, kDSA, kSDA, kSU, kSUA, kUSA, kCIhyd,
     KA, A0, fATP, N, M]
tau = 6.0
U0 = 3.5
p = [p_rest..., tau, U0];

For performance the right-hand-side functions are written with in-place assignment of the derivatives,
which necessarily involves array mutating.
AD doesn't like array mutating. We can use `Zygote.Buffer` to bypass (which creates
immutable copy of `dx`)

In [75]:
function kaiabc_growing_out(x, p)
    # Buffer bypasses the limitation of mutable arrays
    # see https://fluxml.ai/Zygote.jl/latest/limitations/
    # and https://discourse.julialang.org/t/how-to-use-initialize-zygote-buffer/87653
    dx = Zygote.Buffer(x)
    kaiabc_growing!(dx, x, p, 0)
    copy(dx)
end

x0 = [1.5, 0, 1.0, 0.5, 0.5, 0]

# jacobian supports list of input vectors
jac1 = jacobian(x -> kaiabc_growing_out(x, p), x0)[1]

6×6 Matrix{Float64}:
 -0.513553    0.234615   0.0         0.0307692   0.0414201   1.27337
  0.346154   -0.505861   0.103846    0.0         0.0576923  -1.38462
  0.0         0.138462  -0.478938    0.346154   -0.0189349  -0.568047
  0.0346154   0.0        0.0923077  -0.659707    0.0201183   0.60355
  0.0         0.0        0.15        0.0        -0.332488    0.239645
  0.0         0.0        0.0         0.15        0.0994083  -0.296689

Numerically compute the Jacobian by finite differentiation.
We'll compare the result with that obtained by AD for a sanity check

In [33]:
using FiniteDiff

In [47]:
jac2 = zeros(6, 6)
FiniteDiff.finite_difference_jacobian!(jac2, (dx, x) -> kaiabc_growing!(dx, x, p, 0), x0)
jac2

6×6 Matrix{Float64}:
 -0.496294    0.234615   0.0         0.0307692   0.0414201   1.27337
  0.346154   -0.488601   0.103846    0.0         0.0576923  -1.38462
  0.0         0.138462  -0.461678    0.346154   -0.0189349  -0.568047
  0.0346154   0.0        0.0923077  -0.642448    0.0201183   0.60355
  0.0         0.0        0.15        0.0        -0.315229    0.239645
  0.0         0.0        0.0         0.15        0.0994083  -0.27943

In [56]:
all(@. abs(jac1 - jac2) < 1e-6)

true

Now, we are certain that our code get the correct Jacobian by doing the chain rule under the hood.

The next steps are clear: Find the fixed point. Get the eigenvalues of the jacobian about the fixed point, and find the complex conjugates

In [10]:
function vec_mag(x, p)
    dx = similar(x)
    kaiabc_growing!(dx, x, p, 0)
    sum(dx.^2)
end

x0 = [3.5, zeros(5)...]
prob = OptimizationProblem(vec_mag, x0, p)
sol = solve(prob, NelderMead())

u: 6-element Vector{Float64}:
 2.0468695778466963
 0.9019173900635465
 0.16233419549953354
 0.13347414494467297
 0.12628430029831872
 0.1283808034604627

In [12]:
dx = similar(sol.u)
kaiabc_growing!(dx, sol.u, p, 0)
dx

6-element Vector{Float64}:
  4.258116454225469e-5
 -2.444431225533117e-5
  2.111928050762754e-5
 -2.047901856545853e-6
 -4.301884941999845e-6
  5.253419708580766e-5

Here's the fixed point! Let's find the eigenvalues about it, and tune the doubling time until
the real part crosses 0

In [14]:
jac = jacobian(f, sol.u)[1]
eigvals(jac)

6-element Vector{ComplexF64}:
  -0.724000245359546 - 0.12965447032803545im
  -0.724000245359546 + 0.12965447032803545im
 -0.4199241208371327 + 0.0im
 -0.1155245300933241 + 0.0im
 0.05927385634204332 - 0.3712380325803743im
 0.05927385634204332 + 0.3712380325803743im

In [87]:
function real_eig(_tau)
    # param
    _p = [p_rest..., _tau, U0]
    
    # find fixed point
    x0 = [3.5, zeros(5)...]
    prob = OptimizationProblem(vec_mag, x0, _p)
    fp = solve(prob, NelderMead(), reltol=1e-8).u
    
    # linearize about the fixed point
    jac = jacobian(x -> kaiabc_growing_out(x, _p), fp)[1]
    
    # eigenvalue and get Re
    ev = eigvals(jac)
    real(ev[6])
end

tau0 = 6.0
prob = OptimizationProblem((x, p) -> real_eig(x[1])^2, [tau0], 0)
sol = solve(prob, NelderMead(), reltol=1e-8)
tau_hopf = sol.u[1]

5.385546875000001

Double-check the real part indead crosses 0

In [88]:
p = [p_rest..., tau_hopf, U0]

# find fixed point
x0 = [3.5, zeros(5)...]
prob = OptimizationProblem(vec_mag, x0, p)
fp = solve(prob, NelderMead(), reltol=1e-8).u
print(fp)

# linearize about the fixed point
jac = jacobian(kaiabc_growing_out, fp)[1]

# eigenvalue and get Re
ev = eigvals(jac)


[1.976941227655834, 0.9379329321953622, 0.18853096062166647, 0.1367241129647187, 0.13832828591135393, 0.12051951769142058]

6-element Vector{ComplexF64}:
   -0.7571322795291402 - 0.13800763117632947im
   -0.7571322795291402 + 0.13800763117632947im
   -0.4371739987975799 + 0.0im
  -0.12870506870482792 + 0.0im
 0.0021527613855660058 - 0.34418804056237645im
 0.0021527613855660058 + 0.34418804056237645im